# Inference on model and store the output for further use
Use for deep ensemble method, run model on test data set and store the output into json file
The original DINO code did not support save_results function

In [1]:
import os, sys
import torch, json
import numpy as np

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops
import pickle

In [11]:
scale_config = "4scale"
checkpoint = "0023"
model_config_path = "config/DINO/DINO_" + scale_config + ".py" # change the path of the model config file
model_checkpoint_path = "ckpts/checkpoint" + checkpoint + "_" + scale_config + ".pth" # change the path of the model checkpoint
results_path = "./results/deep_ensemble/" + "DINO_" + checkpoint + "_" + scale_config + ".pkl"
thershold = 0.3 # set a thershold
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 

## Load data set

In [12]:
args.dataset_file = 'coco'
args.coco_path = "../coco/" # the path of coco
args.fix_size = False

dataset_val = build_dataset(image_set='val', args=args)

data_aug_params: {
  "scales": [
    480,
    512,
    544,
    576,
    608,
    640,
    672,
    704,
    736,
    768,
    800
  ],
  "max_size": 1333,
  "scales2_resize": [
    400,
    500,
    600
  ],
  "scales2_crop": [
    384,
    600
  ]
}
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!


## Load model

In [13]:
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

In [ ]:
output_dic = {'pred_logits', 'pred_boxes'}
results = {}
with torch.no_grad():
    for idx in range(len(dataset_val)):
        test_result = {}
        output = model.cuda()(dataset_val[idx][0][None].cuda())
        for k in output_dic:
            test_result[k] = output[k].cpu().numpy()
        output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]
        boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
        output['scores'] = output['scores'].cpu().numpy()
        output['labels'] = output['labels'].cpu().numpy()
        output['boxes'] = boxes.cpu().numpy()
        results[idx] = test_result
        if idx % 100 == 0:
            print(f"Complete test data {idx}")

with open(results_path, "wb") as outfile:
     pickle.dump(results, outfile)
print(f"Complete save results into {results_path}")

Complete test data 0
Complete test data 100
Complete test data 200
Complete test data 300
Complete test data 400
Complete test data 500
Complete test data 600
Complete test data 700


In [36]:
import pickle
results_path = "./results/deep_ensemble/" + "DINO_" + checkpoint + "_" + scale_config + ".pkl"
with open(results_path, "wb") as outfile:
     pickle.dump(results, outfile)

In [43]:
dataset_val[0][1]

{'boxes': tensor([[0.3896, 0.4161, 0.0386, 0.1631],
         [0.1276, 0.5052, 0.2333, 0.2227],
         [0.9342, 0.5835, 0.1271, 0.1848],
         [0.6047, 0.6325, 0.0875, 0.2414],
         [0.5025, 0.6273, 0.0966, 0.2312],
         [0.6692, 0.6190, 0.0471, 0.1910],
         [0.5128, 0.5283, 0.0337, 0.0272],
         [0.6864, 0.5320, 0.0829, 0.3240],
         [0.6125, 0.4462, 0.0236, 0.0839],
         [0.8119, 0.5017, 0.0230, 0.0375],
         [0.7863, 0.5364, 0.0317, 0.2542],
         [0.9562, 0.7717, 0.0224, 0.1073],
         [0.9682, 0.7781, 0.0201, 0.1090],
         [0.7106, 0.3100, 0.0218, 0.0514],
         [0.8866, 0.8316, 0.0573, 0.2105],
         [0.5569, 0.5167, 0.0178, 0.0529],
         [0.6517, 0.5288, 0.0150, 0.0294],
         [0.3880, 0.4784, 0.0222, 0.0414],
         [0.5338, 0.4879, 0.0152, 0.0393],
         [0.6000, 0.6471, 0.1962, 0.2088]]),
 'labels': tensor([64, 72, 72, 62, 62, 62, 62,  1,  1, 78, 82, 84, 84, 85, 86, 86, 62, 86,
         86, 67]),
 'image_id': tensor

In [29]:
with torch.no_grad():
    output = model.cuda()(dataset_val[0][0][None].cuda())
    t = output['dn_meta'].cpu().numpy()

AttributeError: 'NoneType' object has no attribute 'cpu'

In [23]:
output.keys()

dict_keys(['pred_logits', 'pred_boxes', 'aux_outputs', 'interm_outputs', 'interm_outputs_for_matching_pre', 'dn_meta'])

In [51]:
output = model.cuda()(dataset_val[0][0][None].cuda())
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

In [52]:
thershold = 0.3 # set a thershold

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

In [17]:
boxes[select_mask].shape

torch.Size([24, 4])

In [22]:
output

[{'scores': tensor([0.9188, 0.8691, 0.8570, 0.8514, 0.7830, 0.6345, 0.5892, 0.5829, 0.5591,
          0.5439, 0.5189, 0.5174, 0.5034, 0.4608, 0.4550, 0.4419, 0.4209, 0.4034,
          0.3904, 0.3604, 0.3416, 0.3370, 0.3052, 0.3032, 0.2983, 0.2700, 0.2568,
          0.2511, 0.2478, 0.2474, 0.2445, 0.2378, 0.2260, 0.2073, 0.2060, 0.1955,
          0.1940, 0.1895, 0.1891, 0.1883, 0.1881, 0.1819, 0.1745, 0.1728, 0.1713,
          0.1701, 0.1700, 0.1675, 0.1666, 0.1662, 0.1647, 0.1643, 0.1599, 0.1553,
          0.1548, 0.1504, 0.1496, 0.1484, 0.1462, 0.1458, 0.1445, 0.1438, 0.1429,
          0.1421, 0.1415, 0.1376, 0.1365, 0.1347, 0.1343, 0.1339, 0.1319, 0.1316,
          0.1311, 0.1311, 0.1296, 0.1293, 0.1273, 0.1271, 0.1270, 0.1255, 0.1245,
          0.1219, 0.1218, 0.1218, 0.1210, 0.1194, 0.1191, 0.1188, 0.1188, 0.1167,
          0.1158, 0.1154, 0.1140, 0.1137, 0.1137, 0.1135, 0.1105, 0.1098, 0.1094,
          0.1091, 0.1075, 0.1075, 0.1070, 0.1066, 0.1058, 0.1051, 0.1050, 0.1036,
      